<a href="https://colab.research.google.com/github/Shrutika-TechSavvy/Google-Colab-Codes/blob/main/Matrix_Multiplication_OpenMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile matrix_mul.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define N 4  // Size of the square matrices

int main() {
int A[N][N], B[N][N], C[N][N];
int i, j, k;

// Initialize matrices A and B
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        A[i][j] = i + j;
        B[i][j] = i - j;
        C[i][j] = 0;
    }
}

// Matrix multiplication using OpenMP
#pragma omp parallel for private(i,j,k) shared(A,B,C)
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        int sum = 0;
        for (k = 0; k < N; k++) {
            sum += A[i][k] * B[k][j];
        }
        C[i][j] = sum;
    }
}

// Print the result
printf("Result matrix C:\n");
for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
        printf("%d ", C[i][j]);
    }
    printf("\n");
}

return 0;

}


Writing matrix_mul.c


In [2]:

!gcc -fopenmp matrix_mul.c -o matrix_mul
!./matrix_mul


Result matrix C:
14 8 2 -4 
20 10 0 -10 
26 12 -2 -16 
32 14 -4 -22 
